In [32]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

!pip install datetime
import datetime

In [56]:
URL = "https://www.mercadolivre.com.br/ofertas?promotion_type=lightning&container_id=MLB779362-1#origin=scut&filter_applied=promotion_type&filter_position=2&is_recommended_domain=false"

data = requests.get(URL).text

In [57]:
type(data)

str

In [58]:
with open('mercadolivreofertadia.html', 'w', encoding='utf-8') as f:
  f.write(data)

In [59]:
#retorna os titulos dos produtos nas Tags
def get_itens_title(doc):
  title_tags = doc.find_all('p', class_='promotion-item__title')
  titles = []
  for tags in title_tags:
    titles.append(tags.text)

  return titles

In [60]:
def get_itens_prices(doc):
  price_tags = doc.find_all('p', class_='promotion-item__price')
  prices = []
  for tag in price_tags:
    prices.append(tags.text.replace('Â',''))

  return prices

In [61]:
def get_doc(url):
  response = requests.get(url)
  doc = BeautifulSoup(response.text, 'html.parser')
  if response.status_code != 200:
    raise Exception("Falha ao Carregar")

  return doc

In [62]:
def scrape_multiple_pages(n):
  titles, prices = [], []

  for page in range(1, n+1):
    doc = get_doc(URL + str(page))
    titles.extend(get_itens_title(doc))
    prices.extend(get_itens_title(doc))

  itens = {
      'TITLES':titles,
      'PRICE':prices
  }

  return pd.DataFrame(itens)

In [63]:
df_mercado_livre_ofer_dia = scrape_multiple_pages(10)

In [64]:
df_mercado_livre_ofer_dia["SCRAPY_DATETIME"] = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
df_mercado_livre_ofer_dia['PRICE'] = df_mercado_livre_ofer_dia['PRICE'].astype(str)
df_mercado_livre_ofer_dia['PRICE'] = df_mercado_livre_ofer_dia['PRICE'].str.extract(r'(\d+[\.,]?\d*)')
df_mercado_livre_ofer_dia

,TITLES,PRICE,SCRAPY_DATETIME
0,Caixa Amplificada Connect Lights Cm-400 Preta ...,400,2024-04-10 17:37:41
1,"Monitor Gamer Ultrawide 25,7'' Ips Full Hd 26w...","25,7",2024-04-10 17:37:41
2,Caixa Amplificada Connect Power Plus CM-550 Pr...,550,2024-04-10 17:37:41
3,Short Praia Masculino Bermuda Verão Academia T...,NaN,2024-04-10 17:37:41
4,Fritadeira Elétrica Airfry Family 4 Litros 150...,4,2024-04-10 17:37:41
...,...,...,...
475,Calça Jeans Feminina Super Lipo Preta Sawary,NaN,2024-04-10 17:37:41
476,Tênis Masculino Neblina Olympikus Novo,NaN,2024-04-10 17:37:41
477,Britânia Fritadeira Air Fryer BFR21P 4L 1500W ...,21,2024-04-10 17:37:41
478,Mala De Viagem Masculina E Feminina Sunside Mé...,NaN,2024-04-10 17:37:41


In [65]:
df_mercado_livre_ofer_dia.dropna(subset=['PRICE'], inplace=True)
df_mercado_livre_ofer_dia

,TITLES,PRICE,SCRAPY_DATETIME
0,Caixa Amplificada Connect Lights Cm-400 Preta ...,400,2024-04-10 17:37:41
1,"Monitor Gamer Ultrawide 25,7'' Ips Full Hd 26w...","25,7",2024-04-10 17:37:41
2,Caixa Amplificada Connect Power Plus CM-550 Pr...,550,2024-04-10 17:37:41
4,Fritadeira Elétrica Airfry Family 4 Litros 150...,4,2024-04-10 17:37:41
5,Chuteira Futsal Masculina Max 1000 Locker Ecok...,1000,2024-04-10 17:37:41
...,...,...,...
470,Kit 10 Calcinhas Feminina Biquíni Microfibra D...,10,2024-04-10 17:37:41
471,Kit Com 10 Calcinhas Infantil Menina Cotton Mi...,10,2024-04-10 17:37:41
472,Olympikus Ultraleve 130g Feminino Adultos,130,2024-04-10 17:37:41
474,Chuteira Society X Crazyfast Messi 4 adidas,4,2024-04-10 17:37:41


In [66]:
# Exporta o DataFrame para um arquivo CSV
df_mercado_livre_ofer_dia.to_csv('mercado_livre_ofer_dia.csv', index=False, encoding='utf-8')